In [6]:
import spacy
import pandas as pd

In [ ]:
ner = spacy.load("en_core_web_sm")
df = pd.read_csv('liar2\lulu_sample.csv')

<>:2: SyntaxWarning: invalid escape sequence '\l'
<>:2: SyntaxWarning: invalid escape sequence '\l'
C:\Users\lucan\AppData\Local\Temp\ipykernel_127184\2075526283.py:2: SyntaxWarning: invalid escape sequence '\l'
  df = pd.read_csv('liar2\lulu_sample.csv')
